# 様々な教師あり学習手法の実装と評価

## データセットの準備

In [ ]:
from sklearn.datasets import make_classification, make_blobs, make_moons, make_circles
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

### 線形分離可能なデータ（2クラス）

In [ ]:
# make_blobs関数を使います。
X, y = make_blobs(
    n_samples=500, # サンプル数
    n_features=2, # 特徴量の数。今回は2変数で。
    centers=[[0, 0], [1, 1]], # 各クラスの中心。この値を中心にガウス分布でデータを生成します。
    cluster_std=[0.4, 0.4], # 各クラスの標準偏差。これが大きいと、データがばらけます。
    random_state=100, # 再現性のための乱数の種。好きな数字をどうぞ。
)
# データをランダムに訓練用とテスト用に分けます。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)


## データを散布図で可視化

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap="viridis", edgecolor="k", marker="o")
# ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap="coolwarm", marker="x") # テストデータも同時に表示したい場合はこの行をコメントアウトしてください。
plt.show()

## モデルの学習

### サポートベクトルマシン(線形カーネル)

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel="linear", C=1.0)
model.fit(X_train, y_train)

## 精度評価と識別境界の表示

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

# テストデータで予測を行います。
y_pred = model.predict(X_test)
# 精度を計算します。
accuracy = accuracy_score(y_test, y_pred)
print(f"精度: {accuracy:.2f}")

# 混同行列を計算します。
cm = confusion_matrix(y_test, y_pred)
plt.matshow(cm, cmap="Blues")  # 混同行列をヒートマップで表示
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(x=j, y=i, s=cm[i, j])

In [ ]:
# 識別境界を描画します。
# 描画範囲を決める。訓練データの最小値と最大値を使います。
xmin, xmax = X_train[:, 0].min(), X_train[:, 0].max() 
ymin, ymax = X_train[:, 1].min(), X_train[:, 1].max() 

# xmin～xmax, ymin～ymaxの範囲で200x200のメッシュグリッドを作成します。
xx, yy = np.meshgrid(np.linspace(xmin, xmax, 200), np.linspace(ymin, ymax, 200))

# メッシュグリッドの各点(200*200=40000サンプル)で予測を行います。
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
# 予測結果を元の形(200x200)に戻します。
Z = Z.reshape(xx.shape)

# 識別境界を描画します。
plt.contourf(xx, yy, Z, alpha=0.8) # 200x200のメッシュグリッドを描画
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap="viridis", edgecolor="k", marker="o") # 訓練データを描画。不要ならコメントアウトしてください。
# plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap="coolwarm", marker="x") # テストデータも同時に表示したい場合はこの行をコメントアウトしてください。
plt.contour(xx, yy, Z, levels=[0], colors="red", linewidths=1) # 識別境界を赤色で描画
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.show()

# 演習

いろいろなデータ、いろいろなモデルを用いて境界がどのように変わるのか試してみてください。

以下に、使えそうなデータの作り方、モデルの作り方を示します。


## データセットの準備（その２）

### 線形分離不可能なデータ（円状に分布）

In [ ]:
X, y = make_circles(
    n_samples=500, # サンプル数
    noise=0.2, # ノイズの大きさ。これが大きいと、データがばらけます。
    factor=0.5, # 内側の円と外側の円の距離。これが大きいと、データがばらけます。
    random_state=100, # 再現性のための乱数の種。好きな数字をどうぞ。
)
# データをランダムに訓練用とテスト用に分けます。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

### 線形分離不可能なデータ（三日月状に分布）

In [ ]:
X, y = make_moons(
    n_samples=500, # サンプル数
    noise=0.2, # ノイズの大きさ。これが大きいと、データがばらけます。
    random_state=100, # 再現性のための乱数の種。好きな数字をどうぞ。
)
# データをランダムに訓練用とテスト用に分けます。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

### 不均衡データの作成

In [ ]:
from sklearn.utils import shuffle
# X_train, y_trainのクラス0のサンプル数を減らします。
percent = 0.1 # クラス0のサンプル数を10%にします。

class0_idx = np.where(y_train == 0)[0] # クラス0データが何番目にあるかを取得
class1_idx = np.where(y_train == 1)[0] # クラス1データが何番目にあるかを取得

n_class0 = int(len(class0_idx) * percent) # クラス0のサンプル数を10%にします。
selected0_idx = np.random.choice(class0_idx, size=n_class0, replace=False) # 10%のサンプルをランダムに選択

new_idx = np.concatenate([selected0_idx, class1_idx]) # 減らしたクラス0のサンプルとクラス1のサンプルを結合
X_train = X_train[new_idx] # 訓練データを更新
y_train = y_train[new_idx] # 訓練データを更新

# シャッフルした方がよさそう
X_train, y_train = shuffle(X_train, y_train)
print(f"クラス0の訓練サンプル数: {len(np.where(y_train == 0)[0])}")
print(f"クラス1の訓練サンプル数: {len(np.where(y_train == 1)[0])}")


## モデルの学習（その２）

### サポートベクトルマシン（ガウスカーネル）

In [ ]:
model = SVC(kernel="rbf", C=1.0)
model.fit(X_train, y_train)

### 決定木

In [ ]:
from sklearn.tree import DecisionTreeClassifier  # 決定木分類器
model = DecisionTreeClassifier(max_depth=3)  # max_depthは木の深さ
# model = DecisionTreeClassifier(criterion="gini", ccp_alpha=0.0) # αを指定して枝刈りする場合
model.fit(X_train, y_train)

### ランダムフォレスト

In [ ]:
from sklearn.ensemble import RandomForestClassifier  # ランダムフォレスト分類器
model = RandomForestClassifier(n_estimators=100, max_depth=3)  # n_estimatorsは木の本数
model.fit(X_train, y_train)

### xgboost

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(
    n_estimators=100,  # 木の数
    max_depth=3,  # 木の深さ
    learning_rate=0.1,  # 学習率
    random_state=100,  # 再現性のための乱数シード
)
model.fit(X_train, y_train)

### ロジスティック回帰


In [ ]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(
    penalty="l2",  # 正則化の種類。l1はLasso、l2はRidge
    C=1.0,  # 正則化の強さ。小さいほど正則化が強くなる
)
model.fit(X_train, y_train)

### ニューラルネット

In [ ]:
# 3層のニューラルネット。KerasとかPyTorchとか使わずにやってみます。
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(
    hidden_layer_sizes=(100, 50), 
    solver="sgd",  # Optimizer。今回は確率的勾配降下法を指定。他には"adam"や"lbfgs"など
    learning_rate_init=0.001,  # 学習率
    max_iter=10000,  # 最大反復回数
) # 他にもたくさんパラメータがあります。詳しくは、https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
model.fit(X_train, y_train)

### k-近傍法
k-近傍法は、推論時に入力データに最も近いk個の訓練データを探し、そのk個のデータのラベルの多数決をとる手法です。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)  # k近傍法。k=5
model.fit(X_train, y_train)

## SMOTEによるOverSampling

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='auto') # 自動でクラスのサンプル数を揃える
X_train, y_train = smote.fit_resample(X_train, y_train) # 訓練データをリサンプリング

## [おまけ] 計算時間を測りたい場合

いくつか方法がありますが、perf_counterを使う方法を紹介します

In [ ]:
import time

# 高精度な計測
start = time.perf_counter() # ストップウォッチ開始

# ここに計測したい処理を書く。サンプルとして円周率を計算する処理を入れています。
pi = 0
for i in range(1000000): # なんか時間かかりそうな処理
    pi += (4 * ((-1) ** i)) / (2 * i + 1) # ライプニッツの公式。詳しくはchatGPTに聞いてください。

end = time.perf_counter() # ストップウォッチ終了
print(f"処理時間: {end - start}秒")
print(f"円周率: {pi:.10f}") # 小数点以下10桁まで表示

